<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

In [2]:
# importing liberaries
import tensorflow as tf
import keras
from tensorflow.python.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

ModuleNotFoundError: No module named 'keras'

In [3]:
# importing data
data = pd.read_csv('data.csv')
data.label = data.label.apply(lambda x: int(x))
x = data[['f1', 'f2']]
y = data.label
print("Data shape: ", x.shape, y.shape)
data.head()

x_train, x_cv, y_train, y_cv = train_test_split(x, y, test_size=0.3, random_state=42)

Data shape:  (20000, 2) (20000,)


In [4]:
print("Train data shape: ", x_train.shape, y_train.shape)
print("CV data shape:    ", x_cv.shape, y_cv.shape)
# data.head()

Train data shape:  (14000, 2) (14000,)
CV data shape:     (6000, 2) (6000,)


In [5]:
# checking wether the data is balanced or not
print(data.label.value_counts())

def lr_schedule(epoch, lr, pre_vcc, cur_val_acc):
    """Helper function to retrieve the scheduled learning rate based on epoch."""
    if cur_val_acc < pre_vcc:
        lr = 0.9 * lr
    if (epoch+1) % 3 == 0:
        lr = 0.95 * lr
    return lr

# callback to find metrics on epoch end
class Metrics(Callback):
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.history={'epoch':[], 'learning_rate':[], 'loss':[],'acc':[], 'val_loss':[], 'val_acc':[], 'auc':[], 'f1_micro':[]}

    def on_epoch_end(self, epoch, logs={}):
        y_hat_pred = np.asarray(self.model.predict(self.x))
        y_hat = np.where(y_hat_pred > 0.5, 1, 0)
        
        self.history['epoch'].append(epoch+1)
        
        # Terminating the training if loss is NaN
        if np.isnan(logs.get('loss', np.nan)):
            print('model stoped training, because loss found to be NaN...')
            sef.model.stop_training = True
        else:
            self.history['loss'].append(logs.get('loss'))
        
        self.history['acc'].append(logs.get('acc'))
        
        if logs.get('val_loss', -1) != -1:
            self.history['val_loss'].append(logs.get('val_loss'))
        
        
        if logs.get('val_acc', -1) != -1:
            self.history['val_acc'].append(logs.get('val_acc'))
         
        # finding auc and micro f1_score
        auc = round(roc_auc_score(self.y, y_hat_pred), 4)
        f1_micro = round(f1_score(self.y, y_hat, average='micro'), 4)
        self.history['auc'].append(auc)
        self.history['f1_micro'].append(f1_micro)
        print('\nauc: {}    f1_micro: {}'.format(auc, f1_micro))
        
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        
        # Get the current learning rate from model's optimizer.
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        self.history['learning_rate'].append(lr)
        
        # Call schedule function to get the scheduled learning rate.
        if epoch != 0:
            scheduled_lr = lr_schedule(epoch, lr, self.history['acc'][-2], self.history['val_acc'][-1])
        else:
            scheduled_lr = lr
        
        # Set the value back to the optimizer before this epoch starts
        tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        print("\nLearning rate is %6.4f." % (scheduled_lr))
        
        # Terminating the training if any of the weight are NaN
        for weights in self.model.get_weights():
            if np.isnan(np.sum(weights)):
                print("model stoped training, because any of the weight found to be NaN...")
                sef.model.stop_training = True
            
        return

0    10000
1    10000
Name: label, dtype: int64


<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>


In [7]:
tf.keras.backend.clear_session()
!rmdir /s /q logs\model1

In [8]:
def create_model():
    return tf.keras.models.Sequential([
        # input layer
        tf.keras.layers.Flatten(input_shape=(2,)),
        # Hidden layars
        tf.keras.layers.Dense(16, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(16, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(8, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(4, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(2, activation='tanh', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        # output layer
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.RandomUniform(0,1))
  ])

binary_model = create_model()
binary_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
                     loss = 'binary_crossentropy',
                     metrics = ['accuracy'])

# 1
metrics_binary = Metrics(x_cv, y_cv)
# 2
filepath="model_save/model1/weights-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
# 3
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1)
# 4
log_dir="logs\\model1\\"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_grads=True)

callbacks_list = [metrics_binary, checkpoint, earlystop, tensorboard_callback]

binary_model.fit(x_train, y_train, epochs=50, validation_data=(x_cv, y_cv), callbacks=callbacks_list)


pd.DataFrame(metrics_binary.history)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 14000 samples, validate on 6000 samples
Epoch 1/50
13888/14000 [============================>.] - ETA: 0s - loss: 0.6983 - acc: 0.5036
auc: 0.4996    f1_micro: 0.5032

Learning rate is 0.1000.

Epoch 00001: val_acc improved from -inf to 0.50317, saving model to model_save/model1/weights-01-0.5032.hdf5
14000/14000 [==============================] - 9s 669us/sample - loss: 0.6984 - acc: 0.5033 - val_loss: 0.7000 - val_acc: 0.5032
Epoch 2/50
13856/14000 [============================>.] - ETA: 0s - loss: 0.6960 - acc: 0.5065
auc: 0.5284    f1_micro: 0.4968

Learning rate is 0.0900.

Epoch 00002: val_acc did not improve from 0.50317
14000/14000 [==============================] - 8s 572us/sample

,epoch,learning_rate,loss,acc,val_loss,val_acc,auc,f1_micro
0,1,0.100000,0.698392,0.503286,0.699961,0.503167,0.4996,0.5032
1,2,0.100000,0.695977,0.506857,0.696202,0.496833,0.5284,0.4968
2,3,0.090000,0.693431,0.516786,0.687407,0.503167,0.5446,0.5032
3,4,0.076950,0.690001,0.522214,0.689233,0.503167,0.5377,0.5032
4,5,0.069255,0.690691,0.522714,0.688394,0.537833,0.5398,0.5378


<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Anamyze your output and training process. 
</pre>

In [6]:
tf.keras.backend.clear_session()
!rmdir /s /q logs\model2

In [7]:
def create_model():
    return tf.keras.models.Sequential([
        # input layer
        tf.keras.layers.Flatten(input_shape=(2,)),
        # Hidden layars
        tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(16, activation='relu', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(8, activation='relu', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(4, activation='relu', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        tf.keras.layers.Dense(2, activation='relu', kernel_initializer=tf.keras.initializers.RandomUniform(0,1)),
        # output layer
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.RandomUniform(0,1))
  ])

binary_model = create_model()
binary_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
                     loss = 'binary_crossentropy',
                     metrics = ['accuracy'])


filepath="model_save/model2/weights-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1)


metrics_binary = Metrics(x_cv, y_cv)


log_dir="logs\\model2\\"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_grads=True)

callbacks_list = [metrics_binary, checkpoint, earlystop, tensorboard_callback]

binary_model.fit(x, y, epochs=50, validation_data=(x_cv, y_cv), callbacks=callbacks_list)


pd.DataFrame(metrics_binary.history)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 20000 samples, validate on 6000 samples
Epoch 1/50
19712/20000 [============================>.] - ETA: 0s - loss: 0.8349 - acc: 0.4995
auc: 0.5    f1_micro: 0.4968

Learning rate is 0.1000.

Epoch 00001: val_acc improved from -inf to 0.49683, saving model to model_save/model2/weights-01-0.4968.hdf5
20000/20000 [==============================] - 9s 459us/sample - loss: 0.8327 - acc: 0.5005 - val_loss: 0.6995 - val_acc: 0.4968
Epoch 2/50
19680/20000 [============================>.] - ETA: 0s - loss: 0.6949 - acc: 0.5046
auc: 0.5    f1_micro: 0.4968

Learning rate is 0.0900.

Epoch 00002: val_acc did not improve from 0.49683
20000/20000 [==============================] - 7s 356us/sample - los

,epoch,learning_rate,loss,acc,val_loss,val_acc,auc,f1_micro
0,1,0.100000,0.832745,0.50055,0.699503,0.496833,0.5,0.4968
1,2,0.100000,0.694909,0.50490,0.703906,0.496833,0.5,0.4968
2,3,0.090000,0.695010,0.50070,0.694375,0.503167,0.5,0.5032
3,4,0.076950,0.694826,0.49840,0.693858,0.503167,0.5,0.5032
4,5,0.076950,0.695361,0.49580,0.693469,0.503167,0.5,0.5032
5,6,0.076950,0.694364,0.49910,0.695017,0.503167,0.5,0.5032
6,7,0.073103,0.695100,0.49840,0.697387,0.503167,0.5,0.5032


<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>

In [ ]:
tf.keras.backend.clear_session()
!rmdir /s /q logs\model3

In [7]:
def create_model():
    return tf.keras.models.Sequential([
        # input layer
        tf.keras.layers.Flatten(input_shape=(2,)),
        # Hidden layars
        tf.keras.layers.Dense(32, activation='relu', kernel_initializer=tf.keras.initializers.he_uniform()),
        tf.keras.layers.Dense(16, activation='relu', kernel_initializer=tf.keras.initializers.he_uniform()),
        tf.keras.layers.Dense(8, activation='relu', kernel_initializer=tf.keras.initializers.he_uniform()),
        tf.keras.layers.Dense(4, activation='relu', kernel_initializer=tf.keras.initializers.he_uniform()),
        tf.keras.layers.Dense(2, activation='relu', kernel_initializer=tf.keras.initializers.he_uniform()),
        # output layer
        tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.he_uniform())
  ])

binary_model = create_model()
binary_model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
                     loss = 'binary_crossentropy',
                     metrics = ['accuracy'])

filepath="model_save/model3/weights-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1)

metrics_binary = Metrics(x_cv, y_cv)


log_dir="logs\\model3\\"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_grads=True)

callbacks_list = [metrics_binary, checkpoint, earlystop, tensorboard_callback]

binary_model.fit(x, y, epochs=50, validation_data=(x_cv, y_cv), callbacks=callbacks_list)


pd.DataFrame(metrics_binary.history)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 20000 samples, validate on 6000 samples
Epoch 1/50
19712/20000 [============================>.] - ETA: 0s - loss: 0.6524 - acc: 0.6187
auc: 0.6987    f1_micro: 0.6333

Learning rate is 0.1000.

Epoch 00001: val_acc improved from -inf to 0.63333, saving model to model_save/model3/weights-01-0.6333.hdf5
20000/20000 [==============================] - 10s 512us/sample - loss: 0.6519 - acc: 0.6191 - val_loss: 0.6386 - val_acc: 0.6333
Epoch 2/50
19872/20000 [============================>.] - ETA: 0s - loss: 0.6338 - acc: 0.6485
auc: 0.7164    f1_micro: 0.6585

Learning rate is 0.1000.

Epoch 00002: val_acc improved from 0.63333 to 0.65850, saving model to model_save/model3/weights-02-0.6585.hdf5
20000/20000 [==============================] - 10s 511us/sample - loss: 0.6342 - acc: 0.6481 - val_loss: 0.6309 - val

,epoch,learning_rate,loss,acc,val_loss,val_acc,auc,f1_micro
0,1,0.1000,0.651884,0.61915,0.638589,0.633333,0.6987,0.6333
1,2,0.1000,0.634159,0.64810,0.630876,0.658500,0.7164,0.6585
2,3,0.1000,0.641033,0.63675,0.632052,0.638667,0.7052,0.6387
3,4,0.0855,0.628830,0.65415,0.643232,0.631333,0.7156,0.6313


<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [9]:
tf.keras.backend.clear_session()
!rmdir /s /q logs\model4

In [10]:
def create_model():
    return tf.keras.models.Sequential([
        # input layer
        tf.keras.layers.Flatten(input_shape=(2,)),
        # Hidden layars
        tf.keras.layers.Dense(32, activation='tanh', kernel_initializer='glorot_uniform'),
        tf.keras.layers.Dense(16, activation='tanh', kernel_initializer='glorot_uniform'),
        tf.keras.layers.Dense(8, activation='tanh', kernel_initializer='glorot_uniform'),
        tf.keras.layers.Dense(4, activation='tanh', kernel_initializer='glorot_uniform'),
#         tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(2, activation='tanh', kernel_initializer='glorot_uniform'),
        # output layer
        tf.keras.layers.Dense(1, activation='tanh', kernel_initializer='glorot_uniform')
  ])

binary_model = create_model()
binary_model.compile(optimizer = 'adam',
                     loss = 'binary_crossentropy',
                     metrics = ['accuracy'])

filepath="model_save/model4/weights-{epoch:02d}-{val_acc:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=1)

metrics_binary = Metrics(x_cv, y_cv)


log_dir="logs\\model4\\"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_grads=True)

callbacks_list = [metrics_binary, checkpoint, earlystop, tensorboard_callback]

binary_model.fit(x, y, epochs=50, validation_data=(x_cv, y_cv), callbacks=callbacks_list)


pd.DataFrame(metrics_binary.history)

Train on 20000 samples, validate on 6000 samples
Epoch 1/50
19936/20000 [============================>.] - ETA: 0s - loss: 0.8505 - acc: 0.6144
auc: 0.7119    f1_micro: 0.6535

Learning rate is 0.0010.

Epoch 00001: val_acc improved from -inf to 0.65350, saving model to model_save/model4/weights-01-0.6535.hdf5
20000/20000 [==============================] - 12s 602us/sample - loss: 0.8499 - acc: 0.6147 - val_loss: 0.6511 - val_acc: 0.6535
Epoch 2/50
19872/20000 [============================>.] - ETA: 0s - loss: 0.6412 - acc: 0.6626
auc: 0.716    f1_micro: 0.6523

Learning rate is 0.0010.

Epoch 00002: val_acc did not improve from 0.65350
20000/20000 [==============================] - 9s 452us/sample - loss: 0.6413 - acc: 0.6622 - val_loss: 0.6374 - val_acc: 0.6523
Epoch 3/50
19776/20000 [============================>.] - ETA: 0s - loss: 0.6250 - acc: 0.6651
auc: 0.7132    f1_micro: 0.6527

Learning rate is 0.0009.

Epoch 00003: val_acc did not improve from 0.65350
20000/20000 [=========

,epoch,learning_rate,loss,acc,val_loss,val_acc,auc,f1_micro
0,1,0.001000,0.849896,0.61465,0.651069,0.653500,0.7119,0.6535
1,2,0.001000,0.641288,0.66225,0.637434,0.652333,0.7160,0.6523
2,3,0.001000,0.624956,0.66505,0.628262,0.652667,0.7132,0.6527
3,4,0.000855,0.614023,0.66575,0.618836,0.664500,0.7174,0.6645
4,5,0.000770,0.609872,0.66600,0.617813,0.662667,0.7170,0.6627
5,6,0.000693,0.607980,0.66905,0.615071,0.663333,0.7243,0.6633
6,7,0.000592,0.604588,0.67185,0.612153,0.664167,0.7237,0.6642
7,8,0.000533,0.603922,0.67380,0.612106,0.665167,0.7240,0.6652
8,9,0.000480,0.603468,0.66900,0.611423,0.663833,0.7247,0.6638
9,10,0.000410,0.602898,0.66995,0.612277,0.663333,0.7235,0.6633
